In [1]:
import os, sys
import shutil
import numpy as np
import pandas as pd
from os.path import join

In [4]:
### Configuration vars
startImage_path = '../task/startImage.jpg'
trialDur = 3000
trialWin = (500, 2500)
calibGrid_path = '../referenceGrids/calibrationGrid.jpg'

# dict to store the pixels/deg visual angle on the calibration grid at various distances
# The calibration grid is 1000px on edge, and 204mm on edge in the real world
pixPerDeg = {'1M': 85.8, '2M': 171.2, '3M': 256.7}

# dict to store the fps of gaze data based on different glasses models
gaze_fps = {'Tobii': 50, 'PL': 60, 'SMI': 60}

### Specify the condition to debug

In [7]:
condition = '103_SMI_1M_0deg'

In [12]:
# parse condition
subj, glasses, distance, offset = condition.split('_')

# store the maximum number of gazepts per trial based on the trial window and glasses sampling Hz
idealMaxGazePts = int((trialWin[1]-trialWin[0])/1000 * gaze_fps[glasses])

### set up inputs/outputs
dataDir = join('../data', condition)
procDir = join(dataDir, 'processed')
calibDir = join(dataDir, 'calibration')

# create calibration output dir
if not os.path.isdir(calibDir):
    os.makedirs(calibDir)

# copy the task log for this condition to the calibration dir
taskLog_fname = join('../data', 'taskLogs', (condition + '_taskLog.txt'))
if os.path.exists(taskLog_fname):
    shutil.copy(taskLog_fname, calibDir)
else:
    print('Could not find task log for: {}'.format(condition))

### find the video frame where the start image appears
if not os.path.exists(join(calibDir, 'startFrame.txt')):
    print('Searching for task start frame...')
    startFrameNum, startFrame = findStartFrame(join(dataDir, 'worldCamera.mp4'))
    cv2.imwrite(join(calibDir, ('startFrame_' + str(startFrameNum).zfill(4) + '.jpg')), startFrame)
    with open(join(calibDir, 'startFrame.txt'), 'w') as f:
        f.write(str(startFrameNum))
else:
    # if already done, just read the text file
    with open(join(calibDir, 'startFrame.txt'), 'r') as f:
        startFrameNum = int(f.read())
        
### find the timestamp of the start image
gazeWorld_df = pd.read_table(join(dataDir, 'gazeData_world.tsv'), sep='\t')
startImage_df = gazeWorld_df[gazeWorld_df.frame_idx == (startFrameNum-1)].iloc[0]
taskStartTime = startImage_df.timestamp

### Load the mapped gaze data, add column with ts relative to the task
gaze_df = pd.read_table(join(procDir, 'gazeData_mapped.tsv'), sep='\t')
gaze_df.loc[:, 'task_ts'] = gaze_df.gaze_ts - taskStartTime

### Load the task log start times
taskLog = pd.read_table(join(calibDir, (condition + '_taskLog.txt')))
calibGridDims = (taskLog.col.max(), taskLog.row.max())

In [13]:
taskLog

,col,row,time
0,5,1,6435
1,3,5,11725
2,3,1,16808
3,1,1,21829
4,5,3,27117
5,5,5,32556
6,1,3,37638
7,1,5,42868
8,3,3,48006


In [19]:
i = 0   # specify which trial you want
# each point is pd.series object with entries for Col, Row, and Time
thisPt = taskLog.iloc[i]

thisTrial = str(i+1)

# get trial start time for this pt
trialStart = thisPt.time

# figure out the index location of this point (counting from L->R, T->B)
ptIdx = (thisPt.row-1) * calibGridDims[0] + thisPt.col

# isolate the gaze data that corresponds to this pt only (return a copy of the gaze_df dataframe)
trialGaze_df = gaze_df[(gaze_df.task_ts > trialStart) & (gaze_df.task_ts <= (trialStart + trialDur))].copy()

# Insert col of timestamps relative to the trial onset
# trialGaze_df.assign(trial_ts = lambda x: x.task_ts - trialStart)
# print(trialGaze_df.columns)
trialGaze_df['trial_ts'] = trialGaze_df.task_ts - trialStart

# isolate the trial data to only those timepoints that fall within the specified analysis window
trialGaze_df = trialGaze_df[(trialGaze_df.trial_ts > trialWin[0]) & (trialGaze_df.trial_ts <= trialWin[1])]




In [20]:
trialGaze_df

,worldFrame,gaze_ts,confidence,world_gazeX,world_gazeY,border_gazeX,border_gazeY,calibGrid_gazeX,calibGrid_gazeY,task_ts,trial_ts
964,593,335048.964,1.0,453.12,294.48,505.0,725.0,835.0,878.0,6936.821,501.821
965,594,335065.591,1.0,453.12,295.92,503.0,732.0,829.0,900.0,6953.448,518.448
966,594,335082.148,1.0,453.12,288.00,501.0,689.0,821.0,766.0,6970.005,535.005
967,594,335098.766,1.0,452.16,288.72,496.0,693.0,806.0,779.0,6986.623,551.623
968,595,335115.328,1.0,453.12,285.84,499.0,676.0,814.0,725.0,7003.185,568.185
969,596,335131.947,1.0,452.16,282.96,495.0,661.0,801.0,679.0,7019.804,584.804
970,596,335148.600,1.0,451.20,285.12,490.0,673.0,786.0,716.0,7036.457,601.457
971,597,335165.081,1.0,454.08,287.28,508.0,683.0,842.0,747.0,7052.938,617.938
972,597,335181.669,1.0,455.04,286.56,513.0,679.0,858.0,734.0,7069.526,634.526
973,598,335198.350,1.0,454.08,285.84,508.0,677.0,842.0,728.0,7086.207,651.207


In [21]:
# add info about the calibraiton point to the dataframe
trialGaze_df['col'] = thisPt.col
trialGaze_df['row'] = thisPt.row
trialGaze_df['ptIdx'] = ptIdx
trialGaze_df['trial'] = thisTrial

# calculate gaze point distance/angle from the ideal location
idealLocation = ((1000/6)*thisPt.col, (1000/6)*thisPt.row)

In [23]:
idealLocation

(833.33333333333326, 166.66666666666666)

In [24]:
a = np.nan

In [26]:
import cv2


In [33]:
vid_path = '../data/103_Tobii_1M_0deg/worldCamera.mp4'

# load the video
vid = cv2.VideoCapture(vid_path)
framesToUse = range(170, 205)

frameCounter = 0
while vid.isOpened():
    ret, frame = vid.read()
    if ret == True:
        frameCounter += 1
        if frameCounter in framesToUse:
            fname = join('../data/103_Tobii_1M_0deg','frame_{}.jpg'.format(str(frameCounter)))
            print(fname)
            cv2.imwrite(fname, frame)

    else:
        vid.release()


../data/103_Tobii_1M_0deg/frame_170.jpg
../data/103_Tobii_1M_0deg/frame_171.jpg
../data/103_Tobii_1M_0deg/frame_172.jpg
../data/103_Tobii_1M_0deg/frame_173.jpg
../data/103_Tobii_1M_0deg/frame_174.jpg
../data/103_Tobii_1M_0deg/frame_175.jpg
../data/103_Tobii_1M_0deg/frame_176.jpg
../data/103_Tobii_1M_0deg/frame_177.jpg
../data/103_Tobii_1M_0deg/frame_178.jpg
../data/103_Tobii_1M_0deg/frame_179.jpg
../data/103_Tobii_1M_0deg/frame_180.jpg
../data/103_Tobii_1M_0deg/frame_181.jpg
../data/103_Tobii_1M_0deg/frame_182.jpg
../data/103_Tobii_1M_0deg/frame_183.jpg
../data/103_Tobii_1M_0deg/frame_184.jpg
../data/103_Tobii_1M_0deg/frame_185.jpg
../data/103_Tobii_1M_0deg/frame_186.jpg
../data/103_Tobii_1M_0deg/frame_187.jpg
../data/103_Tobii_1M_0deg/frame_188.jpg
../data/103_Tobii_1M_0deg/frame_189.jpg
../data/103_Tobii_1M_0deg/frame_190.jpg
../data/103_Tobii_1M_0deg/frame_191.jpg
../data/103_Tobii_1M_0deg/frame_192.jpg
../data/103_Tobii_1M_0deg/frame_193.jpg
../data/103_Tobii_1M_0deg/frame_194.jpg


In [27]:
framesToUse = range(170, 205)
176 in framesToUse

True